In [2]:
import json, os
from azureml.core import Workspace
import base64, io
from PIL import Image
import numpy as np
import azure.cognitiveservices.speech as speechsdk
from utils import authenticate_client, entity_linking, key_phrase_extraction, draw_image

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 2.9.2 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 2.9.2 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (cryptography 2.9.2 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml

In [5]:
#performs one-shot speech recognition from the default microphone
# <SpeechRecognitionWithMicrophone>
speech_config = speechsdk.SpeechConfig(subscription=os.environ["SPEECH_API_KEY"], region="eastus")
# The default language is "en-us".
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

#Steup client for Text Analytics
client = authenticate_client(endpoint = os.environ["COGSVC_ENDPOINT_MP"], 
                                key = os.environ["COGSVC_API_KEY_MP"]
                            )   
print("Say something...")
result = speech_recognizer.recognize_once()

# Check the result
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
    key_phrase_extraction(client, result.text)
    entity_linking(client, result.text)
    draw_image(result.text)
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized")
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))
    

Say something...
Recognized: The Eiffel Tower is landing on the moon.
Key Phrases:
	 The Eiffel Tower
	 moon
Linked Entities:

	Name:  Eiffel Tower 	Id:  Eiffel Tower 	Url:  https://en.wikipedia.org/wiki/Eiffel_Tower 
	Data Source:  Wikipedia
	Matches:
		Text: Eiffel Tower
